## Run GCN Model

In [1]:
%load_ext autoreload

%autoreload 2

In [ ]:
import torch
import numpy as np

In [ ]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    mps=False

else:
    mps_device = torch.device("mps")
    mps=True

    print("MPS is available and enabled on device: {}".format(mps_device))

Load X-chromosome data

In [ ]:
from module4_gcn.dataset import GPS_GCN_Dataset
from gcn_model_classes import *
from torch.utils.data import DataLoader

windowed_data = "X_sampled_data_31.npy"
labels = "X_sampled_labels_31.npy"

# Initialize the Dataset
data = GPS_GCN_Dataset(windowed_data, labels, device=mps_device)

train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

# Initialize the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=28, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=28, shuffle=True)


Train/test split

Instantiate model components

Train model